In [1]:
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata
import matplotlib.pyplot as plt

import geopandas as gpd
import geojson
import keplergl
import json
import openpyxl
import geopy
from shapely.geometry import Polygon, Point, MultiPolygon

%matplotlib inline

/Users/rohun/opt/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [12]:
properties = pd.read_csv('ad36_421Properties_TaxClass2_withTaxData.csv')

In [13]:
properties.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'the_geom', 'cnstrct_yr', 'bin',
       'lstmoddate', 'lststatype', 'doitt_id', 'heightroof', 'feat_code',
       'groundelev', 'shape_area', 'shape_len', 'base_bbl', 'mpluto_bbl',
       'geomsource', 'name', 'Latitude', 'Longitude', 'index_right',
       'Shape_Leng', 'Shape_Area', 'AssemDist', 'BOROUGH', 'NEIGHBORHOOD',
       'BUILDING CLASS CATEGORY', 'TAX CLASS', 'BLOCK', 'LOT',
       'BUILDING CLASS', 'ADDRESS', 'ZIP CODE', 'RESIDENTIAL UNITS',
       'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET',
       'GROSS SQUARE FEET', 'YEAR BUILT', 'BBL', 'Rent Stabilized', 'Units',
       'Due Date', 'RS ID', 'To Drop', 'Total Charge', 'Date'],
      dtype='object')

In [18]:
properties['Total Charge'] = properties['Total Charge'].apply(lambda x: float(x.replace('$', '').replace(',','')))

In [19]:
properties.head()

,Unnamed: 0,Unnamed: 0.1,the_geom,cnstrct_yr,bin,lstmoddate,lststatype,doitt_id,heightroof,feat_code,...,GROSS SQUARE FEET,YEAR BUILT,BBL,Rent Stabilized,Units,Due Date,RS ID,To Drop,Total Charge,Date
0,0,0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",2009.0,4006371,2017-08-22T00:00:00.000Z,Constructed,1113607,100.194260,2100,...,33500,2009,4005520022,Rent Stabilization Fee- Chg,30,01/01/2022,41089700.0,NaN,600.0,20211120
1,1,0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",2009.0,4006371,2017-08-22T00:00:00.000Z,Constructed,1113607,100.194260,2100,...,33500,2009,4005520022,Rent Stabilization Fee- Chg,30,01/01/2021,41089700.0,NaN,600.0,20201121
2,2,0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",2009.0,4006371,2017-08-22T00:00:00.000Z,Constructed,1113607,100.194260,2100,...,33500,2009,4005520022,Rent Stabilization Fee- Chg,30,01/01/2020,41089700.0,NaN,300.0,20191119
3,3,7,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",2008.0,4005656,2017-08-22T00:00:00.000Z,Constructed,1109509,53.890919,2100,...,39745,2008,4005150031,Rent Stabilization Fee- Chg,36,01/01/2022,42909800.0,NaN,720.0,20211120
4,4,7,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",2008.0,4005656,2017-08-22T00:00:00.000Z,Constructed,1109509,53.890919,2100,...,39745,2008,4005150031,Rent Stabilization Fee - Chg,0,NaN,NaN,NaN,0.0,20201121


In [32]:
properties.drop('the_geom', inplace = True, axis = 1)

In [33]:
kepler_map = keplergl.KeplerGl(height=700)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [41]:
kepler_map.add_data(data = properties, name = 'Units')


In [44]:
kepler_map

KeplerGl(data={'Units': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [46]:

kepler_map.save_to_html(data={'data_1': properties}, file_name='TaxClass2Map.html')

Map saved to TaxClass2Map.html!


In [23]:
#try another map
properties_grouped = properties.groupby(['ADDRESS', 'the_geom', 'Latitude', 'Longitude']).sum('Units', 'Total Charge')

In [24]:
properties_grouped.head()

Unnamed: 0  \
ADDRESS         the_geom                                           Latitude  Longitude                
12-15 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766971 -73.933355         195   
                {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767002 -73.934058         159   
12-26 30 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.770865 -73.930605          12   
14-08 31 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767913 -73.931804         690   
14-19 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766274 -73.933080         234   

                                                                                         Unnamed: 0.1  \
ADDRESS         the_geom                                           Latitude  Longitude                  
12-15 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766971 -73.933355           132   
                {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767002 -73.934058           126   
12-26 30 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.770865 -73.930605            21   
14-08 31 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767913 -73.931804           666   
14-19 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766274 -73.933080           160   

                                                                                         cnstrct_yr  \
ADDRESS         the_geom                                           Latitude  Longitude                
12-15 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766971 -73.933355     12066.0   
                {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767002 -73.934058     12066.0   
12-26 30 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.770865 -73.930605      6024.0   
14-08 31 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767913 -73.931804      6009.0   
14-19 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766274 -73.933080      8072.0   

                                                                                              bin  \
ADDRESS         the_geom                                           Latitude  Longitude              
12-15 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766971 -73.933355  27646986   
                {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767002 -73.934058  27253332   
12-26 30 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.770865 -73.930605  12016968   
14-08 31 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767913 -73.931804  12017664   
14-19 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766274 -73.933080  18470980   

                                                                                                                                lstmoddate  \
ADDRESS         the_geom                                           Latitude  Longitude                                                       
12-15 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766971 -73.933355  2017-08-22T00:00:00.000Z2017-08-22T00:00:00.00...   
                {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767002 -73.934058  2017-08-22T00:00:00.000Z2017-08-22T00:00:00.00...   
12-26 30 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.770865 -73.930605  2017-08-22T00:00:00.000Z2017-08-22T00:00:00.00...   
14-08 31 AVENUE {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.767913 -73.931804  2017-08-22T00:00:00.000Z2017-08-22T00:00:00.00...   
14-19 BROADWAY  {'type': 'MultiPolygon', 'coordinates': [[[[-73... 40.766274 -73.933080  2018-12-21T00:00:00.000Z2018-12-21T00:00:00.00...   

                                                                                                                                lststatype  \
ADDRESS         the_geom                                        